In [0]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
warnings.simplefilter("ignore")

In [0]:
data = pd.read_csv('data.csv')

def validate(f, X, Y, CV, modelo, dataset):
    global Dataframe_final
    results = cross_validate(f, X, Y, cv = CV, scoring='accuracy')
    acuracia = results['test_score'].mean()
    results = cross_validate(f, X, Y, cv = CV, scoring='precision')
    precision = results['test_score'].mean()
    results = cross_validate(f, X, Y, cv = CV, scoring='recall')
    recall = results['test_score'].mean()
    array_visualizacao = [[modelo,acuracia,precision,recall,dataset]]
    cols = ['Modelo','Acurácia','Precisão','Recall','Dataset']
    visualizacao = pd.DataFrame(array_visualizacao, columns = cols)
    Dataframe_final = Dataframe_final.append(visualizacao)
    visualizacao.head()

def validateC(f, X, Y, CV, modelo, dataset):
    global Dataframe_final
    results = cross_validate(f, X, Y, cv = CV, scoring='accuracy')
    acuracia = results['test_score'].mean()
    results = cross_validate(f, X, Y, cv = CV, scoring='precision')
    precision = results['test_score'].mean()
    results = cross_validate(f, X, Y, cv = CV, scoring='recall')
    recall = results['test_score'].mean()
    C = f.C
    Grau = f.degree
    Kernel = f.kernel
    array_visualizacao = [[modelo,acuracia,precision,recall,dataset,C,Grau,Kernel]]
    cols = ['Modelo','Acurácia','Precisão','Recall','Dataset', 'C','Grau','Kernel']
    visualizacao = pd.DataFrame(array_visualizacao, columns = cols)
    Dataframe_final =  Dataframe_final.append(visualizacao)
    visualizacao.head()

In [0]:
map = {"M":1, "B":0}
data['diagnosis'] = data['diagnosis'].replace(map)
data = data.drop(columns = 'Unnamed: 32')

In [0]:
from sklearn.decomposition import PCA

In [0]:
data = data.drop(columns='id')
data.head()

In [0]:
X = data.drop(columns='diagnosis')
Y = data[['diagnosis']]

In [0]:
pca = PCA(n_components=2)
pca.fit(X);

In [0]:
projected = pca.fit_transform(X)
projected.data.shape

In [0]:
X_pca2 = pd.DataFrame(data=projected)
X_pca2['diagnosis'] = data['diagnosis']
X_pca2['PC1'] = X_pca2[0]
X_pca2['PC2'] = X_pca2[1]
X_pca2.head()

In [0]:
Dataset_pca2 = X_pca2[['PC1','PC2','diagnosis']]
Dataset_pca2.head()

In [0]:
X_pca2 = X_pca2.drop(columns='diagnosis');
Y_pca2 = Dataset_pca2[['diagnosis']];

# Datasets concluidos : Dataset_pca2(X_pca2, Y_pca2) , data(X,Y)

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [0]:
lda = LDA()
X_lda = lda.fit_transform(X, Y)

In [0]:
Dataset_lda = pd.DataFrame(X_lda)
Dataset_lda[['diagnosis']] = Y
Dataset_lda.head()

# Datasets concluidos : Dataset_pca2(X_pca2, Y_pca2) , data(X,Y), Dataset_lda

In [0]:
from sklearn.preprocessing import StandardScaler
import time

In [0]:
scaler = StandardScaler().fit(X)
Dataset_puro_normalizado = pd.DataFrame(data)

## Puro
X_SVM_Puro = scaler.transform(X)

## PCA2
scaler = StandardScaler().fit(X_pca2)
X_SVM_PCA2 = scaler.transform(X_pca2)

## LDA
X_lda = Dataset_lda.drop(columns='diagnosis')
scaler = StandardScaler().fit(X_lda)
X_SVM_lda = scaler.transform(X_lda)

X_SVM = [
    (X_SVM_Puro, Y, 'Puro'),
    (X_SVM_PCA2, Y, 'PCA2'),
    (X_SVM_lda, Y, 'LDA'),
]

In [0]:
Dataframe_final = pd.DataFrame(columns = ['Modelo','Acurácia','Precisão','Recall','Dataset','Tempo'])

In [0]:
from sklearn.svm import LinearSVC, SVC

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
SEED = 10
np.random.seed(SEED)
# Aleatoriedade no cross validate
# CV = KFold(n_splits = 10, shuffle = True)

CV = StratifiedKFold(n_splits = 10, shuffle = True, random_state=SEED) #estratificando, fazendo cross validation e escolhendo de forma alea

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
param_grid_linearSVC = {'C':[0.01,0.1,1,10]}
grid = GridSearchCV(LinearSVC(), param_grid=param_grid_linearSVC, cv=CV, scoring='recall')
grid.fit(X_SVM_Puro, Y)

linearsvc = grid.best_estimator_
modelo = 'Linear SVC'
dataset = 'Puro'

validate(linearsvc, X_SVM_Puro, Y, CV, modelo, dataset)

In [0]:
grid.best_estimator_

In [0]:
param_grid_linearSVC = {'C':[0.01,0.1,1,10]}
grid = GridSearchCV(LinearSVC(), param_grid=param_grid_linearSVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_

modelo = 'Linear SVC'
dataset = 'PCA2'

validate(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_linearSVC = {'C':[0.01,0.1,1,10,25,50,100]}
grid = GridSearchCV(LinearSVC(), param_grid=param_grid_linearSVC, cv=CV, scoring='recall')
grid.fit(X_SVM_lda, Y)

linearsvc = grid.best_estimator_
modelo = 'Linear SVC'
dataset = 'LDA'

validate(linearsvc, X_SVM_lda, Y, CV, modelo, dataset)

In [0]:
grid.best_estimator_

In [0]:
param_grid_SVC = {'C':[0.01,0.1,1,10],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_Puro, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'Puro'

validate(linearsvc, X_SVM_Puro, Y, CV, modelo, dataset)

In [0]:
grid.best_estimator_

In [0]:
param_grid_SVC = {'C':[1],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
grid.best_estimator_

In [0]:
param_grid_SVC = {'C':[10],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'PCA2'

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_SVC = {'C':[100],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'PCA2'

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_SVC = {'C':[1000],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'PCA2'

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_SVC = {'C':[10000],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'PCA2'

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_SVC = {'C':[100000],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'PCA2'

validateC(linearsvc, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
param_grid_SVC = {'C':[0.01,0.1,1,10,25,50,100],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[1,2,3,4]}
grid = GridSearchCV(SVC(), param_grid=param_grid_SVC, cv=CV, scoring='recall')
grid.fit(X_SVM_lda, Y)

linearsvc = grid.best_estimator_
modelo = 'SVC'
dataset = 'LDA'

validateC(linearsvc, X_SVM_lda, Y, CV, modelo, dataset)

In [0]:
# Para slides : Normalização(pq fizemos), porque nao otimizamos os hiperparametros?
# (separação linear quase perfeita, nao precisaria mexer no coeficiente de regularização), 

# UTilizamos RBF e não funcão de nucleo porque elas sao muito parecidas e a função de nucleo
# tem como graça a possibilidade de utilizacao em datasets nao vetoriais, que nao é nosso caso

# Nosso dataset é bom utilizar o SVM pelo baixo numero de registros

# Por que utilizamos C? Porque o problema nao é linearmente separavel

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
# param_grid_MLP = {'hidden_layer_sizes':[2,5,10,25,50,100], 'activation':['logistic', 'tanh', 'relu'],
#                   'alpha':[0.0001,0.001,0.01,0.1], 'learning_rate_init' : [0.001,0.01,0.1, 1.0]}
param_grid_MLP = {'hidden_layer_sizes':[25,50,75,100], 'activation':['logistic', 'tanh', 'relu'],
                  'alpha':[0.001], 'learning_rate_init' : [0.1, 1.0]}
grid = GridSearchCV(MLPClassifier(random_state=SEED), param_grid=param_grid_MLP, cv=CV, scoring='recall')
grid.fit(X_SVM_Puro, Y)

mlp = grid.best_estimator_
modelo = 'MLP'
dataset = 'Puro'

validate(mlp, X_SVM_Puro, Y, CV, modelo, dataset)

In [0]:
# {'activation': 'tanh',
#  'alpha': 0.001,
#  'hidden_layer_sizes': 50,
#  'learning_rate_init': 0.1}

In [0]:
param_grid_MLP = {'hidden_layer_sizes':[2,5,10,25,50,100], 'activation':['logistic', 'tanh', 'relu'],
                  'alpha':[0.0001,0.001,0.01,0.1], 'learning_rate_init' : [0.001,0.01,0.1]}
grid = GridSearchCV(MLPClassifier(random_state=SEED), param_grid=param_grid_MLP, cv=CV, scoring='recall')
grid.fit(X_SVM_PCA2, Y)

mlp = grid.best_estimator_
modelo = 'MLP'
dataset = 'PCA2'
validate(mlp, X_SVM_PCA2, Y, CV, modelo, dataset)

In [0]:
# {'activation': 'logistic',
#  'alpha': 0.001,
#  'hidden_layer_sizes': 25,
#  'learning_rate_init': 0.1}

In [0]:
param_grid_MLP = {'hidden_layer_sizes':[2,5,10,25,50,100], 'activation':['logistic', 'tanh', 'relu'],
                  'alpha':[0.0001,0.001,0.01,0.1], 'learning_rate_init' : [0.001,0.01,0.1]}
grid = GridSearchCV(MLPClassifier(random_state=SEED), param_grid=param_grid_MLP, cv=CV, scoring='recall')
grid.fit(X_SVM_lda, Y)

mlp = grid.best_estimator_
modelo = 'MLP'
dataset = 'LDA'

validate(mlp, X_SVM_lda, Y, CV, modelo, dataset)

In [0]:
# {'activation': 'tanh',
#  'alpha': 0.01,
#  'hidden_layer_sizes': 2,
#  'learning_rate_init': 0.1}

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
params = {
    'algorithm' : "SAMME",
    'n_estimators' : 100
}

for x,y,dataset in X_SVM:
    ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), **params)
    ada.fit(x, y);

    validate(ada, x, y, CV, 'AdaBoost', dataset)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
for x,y,dataset in X_SVM:
    grad = GradientBoostingClassifier(n_estimators=100)
    grad.fit(x, y);

    validate(grad, x, y, CV, 'GradientBoost', dataset)

In [0]:
Dataframe_final

In [0]:
Dataframe_final = pd.DataFrame(columns = ['Modelo','Acurácia','Precisão','Recall','Dataset','Tempo'])